In [1]:
import pandas as pd
import time

In [2]:
df = pd.read_csv("../csv/2019_crossref_plus_pubmed_temp.csv", index_col=0)
df.shape

(10501, 6)

In [3]:
df.head()

,date_published,authors,journal_title,article_title,all_num_citations_crossref,all_num_references_crossref
0,"[2019, 3]","['Huda, Saif', 'Whittam, Dan', 'Bhojak, Manees...",Clinical Medicine,Neuromyelitis optica spectrum disorders,83.0,60.0
1,"[2019, 1, 4]","['Mann, N.', 'King, T.', 'Murphy, R.']",Clinical and Experimental Dermatology,Review of primary and secondary erythromelalgia,24.0,29.0
2,"[2019, 2]","['Lovgren, Marie-Louise', 'Scarisbrick, Julia ...",Chinese Clinical Oncology,Update on skin directed therapies in mycosis f...,15.0,0.0
3,"[2019, 3]","['Das, Suparna', 'Mitchell, Patrick', 'Ross, N...",World Neurosurgery,Decompressive Hemicraniectomy in the Treatment...,28.0,40.0
4,"[2019, 1, 18]","['Ahmad, Gaity', 'Baker, Jade', 'Finnerty, Joh...",Cochrane Database of Systematic Reviews,Laparoscopic entry techniques,20.0,125.0


In [9]:
df.isnull().sum()

pmid                  0
doi                   0
affiliations          0
abstract           2784
authors               0
journal title       189
article title       189
num_times_cited     189
num_references      189
dtype: int64

In [ ]:
email_address = "yiwench@gmail.com"

def get_info_from_unpaywall(df = df):
    start_time = time.ctime()
    headers = {
        'User-Agent': f'NHS_OA (https://github.com/yiwen-h/nhs_oa/; mailto:{email_address})',
        'From': f'{email_address}'
    }
    all_date_published_upw = []
    all_is_oa = []
    all_oa_status = []
    all_oa_locations = []
    all_is_paratext = []

    for i in range(20):
        doi = df.iloc[i].doi
        url = f"https://api.unpaywall.org/v2/{doi}?email=yiwench@gmail.com"
        response = requests.get(url, headers=headers)
        try:
            response = response.json()
            metadata = response.get('message')
            # get publication date
            date_as_list = metadata.get('published').get('date-parts')[0]
            if len(date_as_list) > 1:
                pub_date = f"{date_as_list[0]}-{date_as_list[1]}"
            else:
                pub_date = f"{date_as_list[0]}"
            all_date_published.append(pub_date)
            # get authors
            author_list = []
            authors_metadata = metadata.get('author')
            for author in authors_metadata:
                author_name = f"{author.get('family')}, {author.get('given')}"
                author_list.append(author_name)
            all_author_list.append(author_list)
            # get journal title
            all_journal_title.append(metadata.get('container-title')[0])
            # get article title
            all_article_title.append(metadata.get('title')[0])
            # get num_citations
            all_num_citations_crossref.append(metadata.get("is-referenced-by-count"))
            # get num_references
            all_num_references_crossref.append(metadata.get("references-count"))
        except:
            all_date_published.append("NaN")
            all_author_list.append(["NaN"])
            all_journal_title.append("NaN")
            all_article_title.append("NaN")
            all_num_citations_crossref.append("NaN")
            all_num_references_crossref.append("NaN")
        
        mini_df_dict = {"date_published" : all_date_published,
                        "authors" : all_author_list,
                        "journal_title": all_journal_title,
                        "article_title": all_article_title,
                        "all_num_citations_crossref" : all_num_citations_crossref,
                        "all_num_references_crossref" :  all_num_references_crossref
                        }
        mini_df = pd.DataFrame(mini_df_dict)
        mini_df.to_csv("csv/2019_crossref_plus_pubmed_temp.csv")
        print(f"{i} / {df.shape[0]} articles metadata obtained from unpaywall")
    df.to_csv("csv/unpaywalltest.csv")
    print(f"Finished. Start time: {start_time}. Finish time: {time.ctime()}")